**Machine Learning for the Quark Model**

Dataset Generation

In [ ]:
# Import libraries
from scipy.optimize import fsolve
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Input experimental data (Mass, n, L, j, J, m_q, m_Q)
data = np.loadtxt('hadrons_data.csv', skiprows=1, delimiter=',')

# Create an array for storing final results
results =[]

In [ ]:
# Run the code for individual quark systems
for s in [1,2,3,4]:

  if s==1:  # (D mesons)
    r = np.arange(0,321,1)
  elif s==2:  # (D_s mesons)
    r = np.arange(321,747,1)
  elif s==3:  # (B mesons)
    r = np.arange(747,857,1)
  elif s==4:  # (B_s mesons)
    r = np.arange(857,986,1)
  
  # Run the code N number of times
  N = 10
  for z in range(N):

    # We generate random 8 indices from our data
    index = np.random.choice(r, 8, replace=False)

    # This is the function to get the 8 equations
    def equations(params):
      a1, a2, b, c, c1, c2, c3, c4 = params

      # We define a function for the mass formula
      def Massformula(vars):

          # These are the variables or inputs
          M, n, L, j, J, m_q, m_Q = vars

          s = 1/2

          L_Sq = 1/2 * (j*(j+1) - L*(L+1) - s*(s+1))

          if L != J:
            Sq_SQ = 1/4

            if L == J+1:
              S_12 = (-2*(J+2))/(2*J+1)
            elif L == J-1:
              S_12 = (-2*(J-1))/(2*J+1)

          elif L == J and J == 0:
            Sq_SQ = -3/4
            S_12 = 0

          elif L == J and J != 0:
            S_12 = 2

            if j == L+1/2:
              Sq_SQ = (-2*L-3)/(8*L+4)
            elif j == L-1/2:
              Sq_SQ = (-2*L+1)/(8*L+4)

          L_SQ = 1/2 * (J*(J+1) - L*(L+1) - 2*s*(s+1)- 2*L_Sq - 2*Sq_SQ)

          H = a1*L_Sq + a2*L_SQ + b*S_12 + c*Sq_SQ

          M_tot = (c1*n + c2*L + c3)**(1/2) + H/(m_q*m_Q) + m_q + m_Q + c4

          return M_tot - M       # This is right hand side minus left hand side, should be minimized
        
      # Here we get the 8 equations we need; we input the data into the mass fomula
      eqs = []
      for i in index:
        eq = Massformula((data[i][0], data[i][1], data[i][2], data[i][3], data[i][4], data[i][5], data[i][6]))
        eqs.append(eq)
      
      # We obtain the equations
      return eqs

    # This solves the equations, with initial guesses in the parenthesis
    a = fsolve(equations, (0.5,0.3,0.5,0.2,1,1,1,1))
    answer = [a[0], a[1], a[2], a[3], a[4], a[5], a[6], a[7]]

    # Store the results into the array
    results.append(answer)

In [ ]:
# Export our results
np.savetxt("training_dataset.csv", np.array(results), delimiter=",")